<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/logic/chatbot_controller.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
OptiBot Chatbot Controller

This module provides the main backend logic for the OptiLine virtual assistant chatbot ("OptiBot").
The chatbot can answer user questions about the OptiLine system, leveraging both generative AI (Gemini) and live data from Firebase when needed.

Functions:
    - ask_optibot(question: str) -> str
        Main entry point. Receives a user question (string) and returns a single English answer as a string.
        Answers can include both real-time data from the system and general knowledge.

Dependencies:
    - google-generativeai (for Gemini API)
    - importnb (for loading other project notebooks as modules)
    - Firebase helper modules (existing: FireBase, user_controller, etc.)
    - Colab secrets (for securely storing Gemini API key)

Note:
    This module is backend-only; it does not include any frontend or UI logic.
    All code comments and docstrings are in English for documentation purposes.
"""

In [ ]:
# ───────────────────────────────────────────────
#  chatbot_controller.py   –  OptiBot back-end
#  Braude College · CIM & Robotics Lab
# ───────────────────────────────────────────────

# ---------- Runtime & project imports ----------
import time
import difflib
import google.generativeai as genai
from google.colab import userdata          # Colab secrets

from importnb import Notebook               # load .ipynb helpers
with Notebook():
    import FireBase as fb                   # Firebase helpers (unused here but kept for future use)
    import user_controller as uc            # leaderboard helper
    import mqqt_sim_indoor as indoor        # indoor sensor stream
    import mqqt_sim_outdoor as outdoor      # outdoor sensor stream

# ---------- Gemini API key ----------
GEMINI_API_KEY = userdata.get("GOOGLE_API_KEY")

# ---------- Knowledge base (official docs) ----------
OPTI_BOT_CONTEXT = """You are OptiBot ... (full text omitted for brevity)"""

# ---------- FAQ tables ----------
FAQ_MAP = {
    "lab":      ["cim lab", "robotics lab", "about the lab"],
    "sensors":  ["available sensors", "sensor types", "what sensors"],
    "points":   ["how to earn points", "score system", "leaderboard points"],
    "simulator":["fault simulator", "fault steps", "skip steps"],
    "app":      ["what is optiline", "about the application"]
}
FAQ_ANSWERS = {
    "lab": (
        "The CIM & Robotics Laboratory was established in 1997 "
        "and is located in room D106. It includes semi-industrial CIM / FMS cells, "
        "robotic arms, CNC simulation, PLCs and more."
    ),
    "sensors": (
        "Integrated sensors: Temperature (°C), Humidity (%), Pressure (hPa), "
        "Dlight (Lux) and Distance (mm). All stream over MQTT."
    ),
    "points": (
        "Points are earned by completing Fault-Simulator challenges. "
        "Difficulty and speed determine the score."
    ),
    "simulator": (
        "All steps in the Fault Simulator must be completed; skipping steps "
        "does not grant partial credit."
    ),
    "app": (
        "OptiLine is a cloud dashboard for the autonomous production line: "
        "live sensors, statistics, MQTT search, Fault Simulator and leaderboard."
    )
}
# quick lookup dicts
key_map  = {kw: topic for topic, lst in FAQ_MAP.items() for kw in lst}
all_keys = list(key_map)

# ---------- General-question whitelist ----------
GENERAL_Q = {
    "time":    ["what time", "current time", "time is it"],
    "date":    ["what date", "what day", "today's date"],
    "who":     ["who are you", "what is optibot", "your name"],
    "help":    ["help", "what can you do", "abilities"],
    "weather": ["weather", "what's the weather", "weather today"]
}
def _match_general(q: str) -> str | None:
    flat = [(tag, kw) for tag, lst in GENERAL_Q.items() for kw in lst]
    phrases = [kw for _, kw in flat]
    hit = difflib.get_close_matches(q, phrases, n=1, cutoff=0.75)
    if not hit:
        return None
    return next(tag for tag, kw in flat if kw == hit[0])

# ---------- Live-sensor configuration ----------
SENSOR_MAP = {
    "temperature": ("Temperature", "°C",  indoor, outdoor),
    "humidity":    ("Humidity",    "%",   indoor, outdoor),
    "pressure":    ("Pressure",    "hPa", indoor, None),
    "dlight":      ("Dlight",      "Lux", None,   outdoor)
}
NORMAL_RANGE = {
    "Temperature": (22, 28),
    "Humidity":    (40, 70),
    "Pressure":    (1005, 1020),
    "Dlight":      (10000, 60000)
}

# ---------- OptiLine-keyword detector ----------
OPTI_KEYWORDS = [
    "optiline", "cim", "robotics lab", "sensor", "temperature",
    "humidity", "pressure", "dlight", "distance", "fault",
    "simulator", "leaderboard", "points", "dashboard",
    "statistics panel", "user directory", "mqtt"
]
def _is_optiline(q: str) -> bool:
    return any(kw in q for kw in OPTI_KEYWORDS)

# ---------- Main entry point ----------
def ask_optibot(message: str, chat_history: list | None) -> tuple[str, list]:
    """
    OptiBot response pipeline:

    1. General-question whitelist          (time / date / who / help / weather)
    2. Live leaderboard top-3
    3. Live sensor value (indoor / outdoor)
    4. FAQ answer from official KB         (exact or fuzzy match)
    5. Free Gemini answer                  (only if NOT OptiLine related)
    6. Refusal                             (OptiLine question but no matching data)
    """
    if chat_history is None:
        chat_history = []
    chat_history.append({"role": "user", "content": message})
    q = message.lower().strip()

    # 1 ─ general
    match = _match_general(q)
    if match:
        if match == "time":
            ans = time.strftime("Current time: %H:%M (Israel).")
        elif match == "date":
            ans = time.strftime("Today is %A, %B %d, %Y.")
        elif match == "who":
            ans = "I am OptiBot, the official assistant for the OptiLine Dashboard."
        elif match == "help":
            ans = (
                "I can show live sensor values, the current leaderboard top-3, "
                "Fault-Simulator guidance, and answers from the official user guide."
            )
        else:                                  # weather
            ans = "I do not have live weather data; please use a weather service."
        chat_history.append({"role": "assistant", "content": ans})
        return "", chat_history

    # 2 ─ leaderboard
    if "leaderboard" in q and any(w in q for w in ("top", "rank", "score")):
        top, _ = uc.get_leaderboard("", top_n=3)
        ans = (
            "Current top-3:\n" +
            "\n".join(f"{r}. {n} – {s} pts" for r, n, s in top)
        ) if top else "Leaderboard data is not available."
        chat_history.append({"role": "assistant", "content": ans})
        return "", chat_history

    # 3 ─ sensor value
    for key, (field, unit, src_in, src_out) in SENSOR_MAP.items():
        if key in q:
            src = src_in if "indoor" in q else src_out if "outdoor" in q else src_out
            if not src:
                ans = "That sensor is not available at the requested location."
            else:
                try:
                    val = next(src.get_live_data_stream(mode="simulation"))[field]
                    lo, hi = NORMAL_RANGE[field]
                    status = "Normal" if lo <= val <= hi else "High" if val > hi else "Low"
                    loc = "indoor" if src is src_in else "outdoor"
                    ans = f"Current {loc} {field.lower()}: {val} {unit} ({status})."
                except Exception as exc:
                    ans = f"Could not retrieve sensor data ({exc})."
            chat_history.append({"role": "assistant", "content": ans})
            return "", chat_history

    # 4 ─ FAQ
    if any(kw in q for kw in key_map):
        topic = next(t for kw, t in key_map.items() if kw in q)
        chat_history.append({"role": "assistant", "content": FAQ_ANSWERS[topic]})
        return "", chat_history
    fuzzy = difflib.get_close_matches(q, all_keys, n=1, cutoff=0.7)
    if fuzzy:
        chat_history.append(
            {"role": "assistant", "content": FAQ_ANSWERS[key_map[fuzzy[0]]]}
        )
        return "", chat_history

    # 5 ─ free Gemini for non-OptiLine topics
    if not _is_optiline(q):
        try:
            if GEMINI_API_KEY:
                genai.configure(api_key=GEMINI_API_KEY)
                model = genai.GenerativeModel("gemini-1.5-flash")
                resp  = model.generate_content(message)
                ans   = resp.text.strip() if hasattr(resp, "text") else str(resp)
            else:
                ans = "Language-model API key is not configured."
        except Exception as exc:
            ans = f"An error occurred: {exc}"
        chat_history.append({"role": "assistant", "content": ans})
        return "", chat_history

    # 6 ─ refusal
    chat_history.append(
        {"role": "assistant",
         "content": (
             "I can answer only according to the official OptiLine documentation. "
             "Please ask about system features, sensors, scoring or faults."
         )}
    )
    return "", chat_history
